In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [19]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type
        if d_type=='train':
            with open('./data/chat_feature_train.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='val':
            with open('./data/chat_feature_val.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        if d_type=='test':
            with open('./data/chat_feature_test.json',"rb") as f1:  
                self.chat_result=json.load(f1)
        
        with open('./LSTM_feature_extractor_Audio_13_ver2/features',"rb") as f2:  
            self.audio=json.load(f2)
        with open('./video_raw_feature_sum_moving_average/video_feature.pickle',"rb") as f2:  
            self.video=pickle.load(f2)
        with open('./label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.chat_result[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

#             window=[]#batch*7(window size)*3(highlight result)
#             for idx in range(23): #7 : window size
#                 s_window=[]
#                 if vframe+idx<len(self.chat_result[game_id]):
#                     s_window+=((self.chat_result[game_id][vframe+idx]))#vframe의 chat
#                     s_window+=(self.audio[game_id][vframe+idx])#vframe의 image
#                     s_window+=self.video[game_id][vframe+idx]
#                 else:
#                     #s_window=[0,0,0]#padding value
#                     s_window=[0]*384
#                 window.append(s_window)
            window=[]
            window+=((self.chat_result[game_id][vframe]))#vframe의 chat
            window+=(self.audio[game_id][vframe])#vframe의 image
            window+=self.video[game_id][vframe]
            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [20]:
train=Mul_data('train')
val=Mul_data('val')

print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([-0.00423759,  0.00849383,  0.01448581, -0.02064631, -0.02426758,
       -0.04851549, -0.04433784, -0.05150938,  0.00209742, -0.00882506,
        0.0166546 , -0.00428606,  0.00834568,  0.00139693, -0.02677757,
        0.02952239, -0.04051007, -0.02685948, -0.043068  , -0.04851913,
        0.02603475, -0.04868141,  0.00577841,  0.01025331,  0.00876541,
        0.00091602, -0.02073224,  0.00888079, -0.06355605, -0.02835641,
        0.01462168,  0.04098507,  0.02122012, -0.02321672, -0.04046787,
        0.03296892, -0.05834012,  0.00234394,  0.00557813, -0.04183179,
        0.03197912,  0.02549705, -0.00770896,  0.01083043, -0.02578666,
       -0.01174926,  0.07748816,  0.01857281,  0.0019242 , -0.01787918,
        0.02091477,  0.02265098,  0.02061003,  0.01535978, -0.02401214,
        0.0086167 ,  0.03011051, -0.00084795, -0.00454742,  0.02914281,
        0.00860303,  0.01394488,  0.01334044,  0.00997945, -0.00854318,
        0.011

In [21]:
train[100][1].shape

(384,)

In [52]:
input_size=384
hidden_size=128
length=7
num_layers=3
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self._lin = nn.Sequential(nn.Linear(in_features=input_size, out_features=hidden_size*2,bias=True),
                                  nn.ReLU(),
                                  nn.Dropout(0.7),
                                  nn.Linear(in_features=hidden_size*2, out_features=hidden_size,bias=True),
                                  nn.ReLU(),
                                  nn.Dropout(0.7),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        out = self._lin(x)

        return out

In [53]:
model=MLP().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [54]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [55]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [56]:
weight_dir='./C_raw+A_raw+V_mov_mlp/'


In [57]:
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(100):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0
epoch 0 train_loss : 0 , val_loss : 0.5593744516372681,p 0.5506719941881584, r 0.6693156732891832, f 0.604224790753288

1
epoch 1 train_loss : 0 , val_loss : 0.48930564522743225,p 0.5884218873909596, r 0.655187637969095, f 0.6200125339461041

2
epoch 2 train_loss : 0 , val_loss : 0.5147227644920349,p 0.6143572621035058, r 0.6498896247240619, f 0.6316241149967818

3
epoch 3 train_loss : 0 , val_loss : 0.49274152517318726,p 0.6308623298033282, r 0.6443708609271523, f 0.6375450475046412

4
epoch 4 train_loss : 0 , val_loss : 0.45681899785995483,p 0.6694175901909927, r 0.626710816777042, f 0.6473606202257439

5
epoch 5 train_loss : 0 , val_loss : 0.4567948579788208,p 0.6625724217844727, r 0.6311258278145695, f 0.6464669304691916

6
epoch 6 train_loss : 0 , val_loss : 0.4881395697593689,p 0.6304770127347291, r 0.6448123620309051, f 0.6375641165557132

7
epoch 7 train_loss : 0 , val_loss : 0.49714037775993347,p 0.6607635694572217, r 0.6342163355408389, f 0.647217841856274

8
epoch 8 train_

epoch 66 train_loss : 0 , val_loss : 0.4500238597393036,p 0.6955008699975143, r 0.6176600441501103, f 0.6542733543785806

67
epoch 67 train_loss : 0 , val_loss : 0.539023756980896,p 0.5417690417690417, r 0.6814569536423841, f 0.6036370746969104

68
epoch 68 train_loss : 0 , val_loss : 0.48652973771095276,p 0.6324382384532761, r 0.6498896247240619, f 0.6410451823625476

69
epoch 69 train_loss : 0 , val_loss : 0.4354514479637146,p 0.6703806870937791, r 0.6375275938189845, f 0.653541525231953

70
epoch 70 train_loss : 0 , val_loss : 0.48325490951538086,p 0.6450828729281768, r 0.6443708609271523, f 0.6447266703478741

71
epoch 71 train_loss : 0 , val_loss : 0.47031453251838684,p 0.6157349896480331, r 0.6565121412803532, f 0.6354700854700854

72
epoch 72 train_loss : 0 , val_loss : 0.48945483565330505,p 0.6323245331616226, r 0.6503311258278146, f 0.6412014365001633

73
epoch 73 train_loss : 0 , val_loss : 0.4765717387199402,p 0.6285106382978723, r 0.6520971302428256, f 0.6400866738894908

7

In [61]:
test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (g,inputs,labels) in enumerate(val_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        output=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,game in enumerate(g):
            if game not in result.keys():
                result[game]=pred[idx].tolist()
            else:
                result[game]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 128 0 tensor(0) tensor(0)
0 0 128 0 tensor(0) tensor(0)
0 0 128 0 tensor(0) tensor(0)
8 1 119 0 tensor(9) tensor(8)
10 0 114 4 tensor(10) tensor(14)
0 0 128 0 tensor(0) tensor(0)
7 1 120 0 tensor(8) tensor(7)
22 18 72 16 tensor(40) tensor(38)
7 11 109 1 tensor(18) tensor(8)
0 0 128 0 tensor(0) tensor(0)
21 7 88 12 tensor(28) tensor(33)
4 11 108 5 tensor(15) tensor(9)
30 10 66 22 tensor(40) tensor(52)
36 17 71 4 tensor(53) tensor(40)
29 6 81 12 tensor(35) tensor(41)
39 2 51 36 tensor(41) tensor(75)
1 0 124 3 tensor(1) tensor(4)
10 0 106 12 tensor(10) tensor(22)
10 2 108 8 tensor(12) tensor(18)
21 4 81 22 tensor(25) tensor(43)
8 11 106 3 tensor(19) tensor(11)
25 18 78 7 tensor(43) tensor(32)
14 8 99 7 tensor(22) tensor(21)
10 21 86 11 tensor(31) tensor(21)
0 13 115 0 tensor(13) tensor(0)
16 10 97 5 tensor(26) tensor(21)
27 6 80 15 tensor(33) tensor(42)
22 13 78 15 tensor(35) tensor(37)
30 18 74 6 tensor(48) tensor(36)
27 0 99 2 tensor(27) tensor(29)
0 0 128 0 tensor(0) 

In [62]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('./label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844294671796427
precision : 0.7181208053691275, recall : 0.6504559270516718, f1 : 0.6826156299840511, accuracy : 0.9030214424951267
102844224145685626
precision : 0.6395348837209303, recall : 0.6606606606606606, f1 : 0.6499261447562777, accuracy : 0.8596802841918295
102844412717407552
precision : 0.7643097643097643, recall : 0.6676470588235294, f1 : 0.7127158555729985, accuracy : 0.9041884816753927
102844235751390657
precision : 0.6061946902654868, recall : 0.6008771929824561, f1 : 0.60352422907489, accuracy : 0.9089068825910931
102844401156069033
precision : 0.6942446043165468, recall : 0.603125, f1 : 0.6454849498327759, accuracy : 0.8855291576673866
102904869420860038
precision : 0.5300546448087432, recall : 0.671280276816609, f1 : 0.5923664122137404, accuracy : 0.8429411764705882
102910307641576395
precision : 0.5946843853820598, recall : 0.5755627009646302, f1 : 0.5849673202614379, accuracy : 0.8479041916167664
102844341905404560
precision : 0.7748091603053435, recall : 0.771863

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571